In [77]:
import psycopg2
import pandas as pd
import geopandas as gpd
from jinja2 import Environment, FileSystemLoader
from config import config

params = config()
conn = psycopg2.connect(**params)

a = !ls ../FINALES/PARCELAS/TORNAVACAS/*shp
parcelas = list(map(lambda x: x.split('/')[-1].split('.')[0], a))
parcelas_dict = []
for parcela in parcelas:
#     for index, value in p.iterrows():
#         display(value.geom)
#     # SUPERFICIE
    
    sup = gpd.read_postgis('''
                SELECT
                    geom,
                    ST_Area(ST_Transform(a.geom, 25830)) / 10000 as sup, 
                    ST_Perimeter(ST_Transform(a.geom, 25830)) as perim,
                    (ST_SummaryStats(St_Union(ST_Clip(b.rast, st_transform(geom, 25830))))).min as pend_min,
                    (ST_SummaryStats(St_Union(ST_Clip(b.rast, st_transform(geom, 25830))))).max as pend_max,
                    (ST_SummaryStats(St_Union(ST_Clip(b.rast, st_transform(geom, 25830))))).mean as pend_mean,
                    (ST_SummaryStats(St_Union(ST_Clip(c.rast, st_transform(geom, 25830))))).min as alt_min,
                    (ST_SummaryStats(St_Union(ST_Clip(c.rast, st_transform(geom, 25830))))).max as alt_max,
                    (ST_SummaryStats(St_Union(ST_Clip(c.rast, st_transform(geom, 25830))))).mean as alt_mean,
                    (ST_SummaryStats(St_Union(ST_Clip(d.rast, st_transform(geom, 25830))))).mean as orient
                FROM 
                    {} a,
                    pendientes25 b,
                    mdt25 c,
                    orientaciones25 d
                GROUP BY
                    a.geom
                '''.format(parcela), conn)
    parcelas_dict.append({'nombre': parcela, 'datos':sup.to_dict(orient='records')})
display(parcelas_dict)
env = Environment(loader=FileSystemLoader('static/evaluacion_previa'))
template =env.get_template('base.html')
output = template.render(
    parcelas = parcelas_dict
)

with open('static/evaluacion_previa/evaluacion_previa.html', 'w') as f:
    f.write(output)

[{'nombre': 'QUEMA_TORNAVACAS_1',
  'datos': [{'geom': <shapely.geometry.multipolygon.MultiPolygon at 0x7f6f9be60910>,
    'sup': 7.901258209096185,
    'perim': 1411.150023451921,
    'pend_min': 10.536633491516113,
    'pend_max': 34.20576477050781,
    'pend_mean': 20.513571449279784,
    'alt_min': 1529.7659912109375,
    'alt_max': 1585.3819580078125,
    'alt_mean': 1563.7407119140626,
    'orient': 152.70846002197266}]},
 {'nombre': 'QUEMA_TORNAVACAS_2_Camino',
  'datos': [{'geom': <shapely.geometry.multipolygon.MultiPolygon at 0x7f6f9c3ecb50>,
    'sup': 3.777421960571417,
    'perim': 1745.4483658668062,
    'pend_min': 18.523210525512695,
    'pend_max': 45.0586051940918,
    'pend_mean': 27.04725272438743,
    'alt_min': 1514.5570068359375,
    'alt_max': 1585.0670166015625,
    'alt_mean': 1547.2925803444602,
    'orient': 173.7119032426314}]},
 {'nombre': 'QUEMA_TORNAVACAS_3_faja_Camino_LA_CAMPANA',
  'datos': [{'geom': <shapely.geometry.multipolygon.MultiPolygon at 0x7f6f